In [148]:
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

In [149]:
def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ')}")
    print('\n')

def print_data_ext_ps(V1: np.complex_, V2: np.complex_):
    """
    Function to print data on the effect of the external phaseshift
    that's used to match the phases of two given elements.
    
    Parameters
    ------
    V1 : element of the auxillary matrix `V`
    V2 : subsequent element of **V1**
    
    ---
    
    Further description:
    ---
    When it comes to the effect of the external phaseshift  `P`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    
    ---
    When it comes to the effect of `exp(i*summa)` in `M`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x-1,y-1]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x+1,y+1]`` ,  ``V2 = V[x,y+1]``
    
    """
    
    eq_stat = (np.angle(V1) == np.angle(V2))
    eq_str = "\nThey are matching!"
    neq_str = "\nThey are NOT matching!\n"

    print('Affected elements:\t',
        "{:.2f}, {:.2f}".format(V1, V2))
    print('Corresponding angles:\t',
        "{:.2f}, {:.2f}".format(np.angle(V1), np.angle(V2)))

    if eq_stat:
        print(eq_str)
    else:
        print(neq_str)
        print('Full length of angles:\n',
            "{}, {}".format(np.angle(V1), np.angle(V2)))
        print('\nTheir difference:\t',
            "{}".format(np.angle(V1) - np.angle(V2)))

In [150]:
U = itf.random_unitary(5)

# Test of algorithm steps

## Odd diagonals

### Finding external phaseshifts
They match the given elements' phases

In [151]:
V = np.conjugate(U.T)
# odd diags: 1,3,5...
m = U.shape[0]
j = 1

x = m-1
y = j-1

P = itf.external_ps(m, j, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

Affected elements:	 0.11+0.44j, 0.18+0.67j
Corresponding angles:	 1.31, 1.31

They are NOT matching!

Full length of angles:
 1.3130203280347816, 1.3130203280347812

Their difference:	 4.440892098500626e-16


### Finding internal phaseshifts
$\delta$ and $\sum$

In [152]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = 0

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	-1.00+0.00j	 -> real: False
angle:	180.0°

Block of M:

[-0.84+1.14e-16j,  0.54+1.77e-16j]
[0.54+1.77e-16j, 0.84-1.14e-16j]




In [153]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -97.91°



## Even diagonals

### Finding external phaseshifts
They match the given elements' phases

In [154]:
# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0

P = itf.external_ps(m, j, V[x,y], V[x-1,y])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

Affected elements:	 -0.14+0.12j, -0.54+0.47j
Corresponding angles:	 2.42, 2.42

They are matching!


### Finding internal phaseshifts
$\delta$ and $\sum$

In [155]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(-V[x-1,y], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = np.angle(V[x+1,y+1]) - np.angle(V[x-1,y+1]*np.cos(delta) - V[x,y+1]*np.sin(delta))

# print('\nangles to get equal by summa:\n',
#       np.angle(V[x+1,y+1]),
#       np.angle(V[x,y+1]*np.cos(delta) - V[x+1,y+1]*np.sin(delta)),'\n',
#       'summa:',summ)

# summ = 0

# might need to change into 'k' dependence
modes = [x-1, x]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])
# print_matrix(M)


# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	1.31-0.00j	 -> real: False
angle:	-1.253598889885402e-15°


In [156]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
print('Elements affected by e^(i*summ):\n')
print_data_ext_ps(V[x+1,y+1], V[x,y+1])

Nulled element:	Re(0.00) Im(-0.00)
abs: 0.00, angle: -24.86°

Elements affected by e^(i*summ):

Affected elements:	 0.21+0.80j, 0.01+0.05j
Corresponding angles:	 1.31, 1.31

They are NOT matching!

Full length of angles:
 1.3130203280347814, 1.3130203280347816

Their difference:	 -2.220446049250313e-16


### Continuation on the chosen diagonal
Without correct $\sum$ it's wrong

In [157]:
x += 1
y += 1

delta = itf.custom_arctan(-V[x-1,y], V[x,y])
print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = 0

modes = [x-1, x]     # initial mode-pairs
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])

# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(x,x+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.07+0.00j	 -> real: False
angle:	1.037795441298518e-14°


In [158]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -105.29°



## Odd diagonal 2.
Diagonal with more than one element

### Finding external phaseshifts
They match the given elements' phases

In [159]:
# odd diags: 1,3,5...
m = U.shape[0]
j = 3

x = m-1
y = j-1

P = itf.external_ps(m, j, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

Affected elements:	 -0.68+0.60j, -0.21+0.19j
Corresponding angles:	 2.42, 2.42

They are matching!


### Finding internal phaseshifts
$\delta$ and $\sum$

In [160]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta))

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	-0.30-0.00j	 -> real: False
angle:	-180.0°

Block of M:

[ 0.28+0.1j , -0.9 -0.31j]
[-0.9 -0.31j, -0.28-0.1j ]




In [161]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(0.00)
abs: 0.00, angle: 156.40°



In [162]:
x -= 1
y -= 1

delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta))

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	-0.54-0.00j	 -> real: False
angle:	-180.0°

Block of M:

[-0.47+0.22j,  0.77-0.37j]
[0.77-0.37j, 0.47-0.22j]




In [163]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.00) Im(0.00)
abs: 0.00, angle: 26.83°



In [168]:
print_matrix(V)

[ 0.04+0.61j,  0.1 -0.26j, -0.11-0.14j,  0.19-0.12j,  0.37+0.58j]
[-0.1 -0.26j,  0.5 +0.65j, -0.11-0.02j,  0.04-0.15j,  0.45+0.13j]
[ 0.66-0.33j,  0.45-0.22j,  0.06-0.08j,  0.1 +0.09j, -0.29+0.29j]
[6.95e-19-2.46e-17j, 5.46e-17+2.76e-17j, 6.26e-01+7.45e-01j,
 5.82e-02-4.18e-02j, 1.25e-01+1.81e-01j]
[ 5.81e-17-2.53e-17j, -5.84e-17+7.51e-17j, -4.34e-17+3.61e-17j,
  8.75e-01-3.71e-01j, -1.16e-01-2.88e-01j]




# Test of module
## <center>TODO</center>
* check if $V$ is a diagonal matrix after the decomposition
---
- extend module to save found phases: $\phi$, $\delta$, $\sum$ $\Rightarrow$ $\theta_1$ & $\theta_2$
- check the function in module that recreates the initial unitary matrix based on the decomposed matrices (probably needs work)
- update the drawing function in the module such that it follows the updated **Clement's** `draw()` function but with sMZI:s
- Finally: sweep through the whole module and finalise *comments*, *documentation*, *variables* and implement possible *optimalisations*
---
* optional: `import numba` for faster execution; optimized machine code at runtime

In [13]:
# including imports again in case someone only wishes
# to run this part of the notebook
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ', suppress_small=True)}")
    print('\n')

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)


U = itf.random_unitary(5)

In [14]:
final_matrix = itf.square_decomposition(U)

In [15]:
# print_matrix(np.conjugate(final_matrix.T))
print_matrix(final_matrix)

[-0.9 +0.23j,  0.09-0.2j , -0.29-0.16j, -0.07+0.13j, -0.35-0.07j]
[ 0.  +0.j  ,  0.45-0.91j, -0.03-0.02j, -0.01+0.01j, -0.04-0.01j]
[ 0.  -0.j  , -0.  -0.j  ,  0.66+0.79j,  0.02-0.02j,  0.05+0.03j]
[ 0.  +0.j  , -0.  -0.j  ,  0.  -0.j  ,  0.78-0.63j,  0.02+0.01j]
[ 0.  -0.j  ,  0.  +0.j  , -0.  -0.j  , -0.  -0.j  ,  0.87+0.55j]




$$\uparrow$$
### <center> The above is only zeroed in the lower triangular part! </center>